In [1]:
import os
import sys

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import numpy as np
import torch
from skimage import measure
from tqdm import tqdm, trange

from core.dataset import ScanNet
from core.integrate import ScalableTSDFVolume

In [3]:
dataset = ScanNet('/home/quanta/Datasets/ScanNet/')
scan_id = 'scene0000_00'
id = dataset.scan_id_list.index(scan_id)
single_instance = dataset[id]['scan_dataset']

In [4]:
save_dir = '/home/quanta/Experiments/feature-instance-fusion/scannet_scene0000_00/'
os.makedirs(save_dir, exist_ok=True)

In [5]:
tsdf_device = 'cuda:1'
tsdf_volume = ScalableTSDFVolume(
    voxel_size=0.015,
    sdf_trunc=0.075,
    margin=0.08,
    device=tsdf_device,
)

In [6]:
tsdf_volume.load(save_dir + 'tsdf/tsdf_vol.pt')

In [7]:
from core.integrate.utils.voxel_to_mesh import get_mesh_from_voxel

In [8]:
verts, faces = get_mesh_from_voxel(tsdf_volume._voxel_hash)

In [9]:
tsdf_volume._voxel_hash.shape

torch.Size([2087398])

In [10]:
verts.shape, faces.shape

(torch.Size([6783756, 3]), torch.Size([4151608, 3]))

In [11]:
import open3d as o3d

draw = o3d.visualization.EV.draw

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [12]:
mesh = o3d.geometry.TriangleMesh(
    vertices=o3d.utility.Vector3dVector(verts.cpu().double().numpy()),
    triangles=o3d.utility.Vector3iVector(faces.cpu().numpy()),
)
mesh.compute_vertex_normals()
draw([mesh])

In [14]:
type(faces.cpu().numpy())

numpy.ndarray